In [1]:
HOST = 'http://localhost:11434'

In [2]:
from visual_info_extractor.ollama.healthcheck import OllamaHealthChecker
from visual_info_extractor.ollama.client import OllamaClient

connection_status = OllamaHealthChecker(host=HOST).check_connection()

if connection_status:
    client = OllamaClient(host='http://localhost:11434')

Ollama is reachable at http://localhost:11434


In [3]:
from visual_info_extractor.io import DataIO
from visual_info_extractor.inferencer import VLMInferencer
from visual_info_extractor.logger import logging
from visual_info_extractor.config import AppConfig
from visual_info_extractor.ollama.healthcheck import OllamaHealthChecker
from visual_info_extractor.ollama.client import OllamaClient

config_path = "../config/gpu.yaml"
config = AppConfig.from_yaml(config_path)
evaluator_model = config.evaluator_model
        

def set_client():
    connection_status = OllamaHealthChecker(host=config.ollama_host).check_connection()
    if connection_status:
        client = OllamaClient(host=config.ollama_host)
    return client

client = set_client()
client.pull_models([evaluator_model])

/home/vm/image-information-2-text/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ollama is reachable at http://localhost:11434
Pulling model: qwen3-vl:30b


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏ 8.5 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  12 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  28 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  44 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  50 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  65 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  79 MB/ 19 GB                  pulling manifest 
pulling b1da6f96a2e4:   0% ▕                  ▏  86 MB/ 19 GB              

In [4]:
def build_judge_prompt(prompt, groundtruth, response):
    return f"""
You are a strict but fair multimodal evaluator.

Your task is to judge whether an assistant response is a correct answer to
the USER PROMPT, given the IMAGE and the GROUND TRUTH.

ROLES:
- The image is the primary source of observable facts.
- The ground truth provides minimum guaranteed facts and must not be contradicted.
- The prompt defines the expected scope, level of detail, and whether interpretation is allowed.

RULES:
1. The response must correctly answer the prompt.
2. The response must be consistent with the ground truth.
3. The response may include additional details beyond the ground truth ONLY if they are clearly visible in the image.
4. Do NOT allow hallucinations, speculation, inferred intent, or unsupported interpretation.
5. If you suspect an hallucination, validate it against the image.
5. Grammatical or spelling differences (e.g., "color" vs "colour", capitalization) do NOT affect correctness.
6. If you are uncertain whether a detail is supported, mark FALSE.

---

PROMPT GIVEN TO THE ASSISTANT:
\"\"\"{prompt}\"\"\"

GROUND TRUTH:
\"\"\"{groundtruth}\"\"\"

IMAGE: received along with this prompt.

ASSISTANT RESPONSE:
\"\"\"{response}\"\"\"

Return ONLY a JSON object with:
- label: TRUE or FALSE
- reason: one short sentence explaining the decision
"""


In [ ]:
import json

def parse_judge_output(content: str):
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {
            "label": "INVALID_JSON",
            "reason": "INVALID_JSON"
        }


In [6]:
import pandas as pd
from pathlib import Path

dfs = []
results_csv_path = "../data/results/1.4-gpu"
for f in Path(results_csv_path).glob("*.csv"):
    tmp = pd.read_csv(f)
    tmp["model"] = f.name.split("_")[1].split(".csv")[0]
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

In [7]:
import glob
len(glob.glob(f"{results_csv_path}/**/*.csv", recursive=True))

6

In [8]:
len(df)

186

In [9]:
df = df[df.image_path != "image_path"]


In [10]:
len(df)

180

In [11]:
evaluator_model

'qwen3-vl:30b'

In [15]:
eval_results = []
value_errors = 0

print("Starting evaluation...")
print("LLM Evaluator Model:", evaluator_model)

for idx, row in df.iterrows():
    print("Evaluating row:", idx)
    print("Model:", row["model"])
    print("Image path:", row["image_path"])
    judge_prompt = build_judge_prompt(
        prompt=row["prompt"],
        groundtruth=row["groundtruth"],
        response=row["response"],
    )

    try:
        content, trace = client.run_chat(
            model=evaluator_model,
            prompt=judge_prompt,
            image_paths=["../" + row["image_path"]],
        )

        judge = parse_judge_output(content)

        eval_results.append({
            "model": row["model"],
            "prompt": row["prompt"],
            "groundtruth": row["groundtruth"],
            "response": row["response"],
            "image_path": row["image_path"],
            "judge_label": judge["label"],
            "judge_reason": judge["reason"],
        })

    except ValueError as e:
        logging.error(f"ValueError for row {idx}: {e}")
        value_errors += 1
        continue

print(f"Total ValueErrors encountered: {value_errors}")
pd.DataFrame(eval_results).to_csv("../data/eval_results_1.4-gpu.csv", index=False)

Starting evaluation...
LLM Evaluator Model: qwen3-vl:30b
Evaluating row: 0
Model: granite3.2-vision:2b
Image path: data/diagram_image_to_text/diagram_image_to_text0.png


2026-01-20 21:03:34,266 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Execution time: 6.79 seconds
Memory used: 0.00 MB
CPU usage change: 0.30%
Evaluating row: 1
Model: granite3.2-vision:2b
Image path: data/diagram_image_to_text/diagram_image_to_text1.png


2026-01-20 21:03:41,256 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Execution time: 6.99 seconds
Memory used: 0.00 MB
CPU usage change: 0.10%
Evaluating row: 2
Model: granite3.2-vision:2b
Image path: data/diagram_image_to_text/diagram_image_to_text10.png


...

Execution time: 7.43 seconds
Memory used: 0.00 MB
CPU usage change: 0.10%
Evaluating row: 184
Model: qwen2.5vl:3b
Image path: data/chart2text/chart2text8.png


2026-01-20 21:43:05,574 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Execution time: 3.23 seconds
Memory used: 0.00 MB
CPU usage change: 0.30%
Evaluating row: 185
Model: qwen2.5vl:3b
Image path: data/chart2text/chart2text9.png


2026-01-20 21:43:12,593 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Execution time: 7.02 seconds
Memory used: 0.00 MB
CPU usage change: 0.10%
Total ValueErrors encountered: 0


In [13]:
df4 = pd.DataFrame(eval_results)

In [14]:
print(df4[['groundtruth', 'response', 'judge_label', 'judge_reason']].to_string())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         groundtruth                                                                                                                                                                                                                                                                                                                                                                    